Notebook de base para armazenar as minhas anotações dos cursos.

Tomarei como base os microdados do enem de 2019. Disponibilizados no [link](https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/enem) . Consultado em 07/01/2021

Por Motivos de Volume de dados e possivel problemas de privacidade eu adicionei a pasta de dados, onde eu vou deixar os dados que vou utilizar nos estudos, no .gitignore desse repositorio. Caso seja do interesse reproduzir o conteudo do que esta ocorrendo aqui, recomendo criar a pasta da mesma forma. 

### Instalação

- A instalação do pyspark varia, dependendo do sistema operacional. Como sempre, recomendo que seja utilizado o linux para a instalação. uma VM já resolve. No meu caso estou rodando o Archlinux

- Para a instalação do Spark, recomendo que seja seguido o tutorial que foi disponibilizado pelo pessoal da sundog Education, para [linux](https://www.tutorialspoint.com/apache_spark/apache_spark_installation.htm), para windows e mac podem ser acompanhado conforme consta no [site](https://sundog-education.com/spark-python/) deles.

- Importante lembrar que, o uso do spark não é recomendado para situações de maquina local, uma vez que, caso seja viavel rodar em um nó unico podemos perfeitamente utilizar o pandas. Eu particularmente gosto de usar o pyspark em uma configuração local para testes menores e preparo para uso em um cluster de produção ou infraestrutura similar.

### importando o pySpark

- Após a instalação, para importar a biblioteca base do pyspark funciona de forma muito simples, com o comando import

``` import pyspark ``` 

- Eu não recomendo que seja utilizado o findmyspark, já que em um ambiente de produção, um AWS EMR por exemplo, não existe a necessidade de ser importado dessa forma, o que pode levar a retrabalhos

In [1]:
import pyspark

### Criando a sessão
- Feito a importação, para executarmos qualquer coisa no spark, precisamos que seja criada uma sessão.
    - A sessão no spark pertence a biblioteca SQL, então para criar precissamos chamar a biblioteca SQL e então o SparkSession.
    - Do SparkSession, precisamos do metodo builder, que funciona para montar a sessão.
    - Após, precisamos do metodo getOrCreate que cria a sessão de fato.
    - A Sessão que criarmos podemos armazenar em uma variavel do python, para podemos executar mais etapas do nosso código.
    - Assim o comando para criarmos a sessão fica:  

In [2]:
sessao = pyspark.sql.SparkSession.builder.getOrCreate()

- Com a Sessão criada, agora podemos executar comandos no spark.

### Importando dados
- Como vamos usar o spark para processos de ETL, vamos precisar fazer a importação de dados. O spark de forma nativa é capaz de ler os principais tipos de dados que são capazes de se comportar bem com volume altos de dados, então CSV, Parquet, avro, json. O uso do jdbc é bem comum para consulta em bancos de dados relacionais. 
- Para importar um csv que esta armazenado de forma local, podemos usar o metodo ```read```  na sessão do spark que criamos previamente. Após o metodo read precisamos informar qual tipo de dado que vamos ler, nesse caso vamos ler um arquivo csv, então usamos a função ```csv``` e apontamos o caminho do arquivo que vamos ler, podendo ele estar dentro de um hdfs, um bucket ou local como é o nosso caso.
- Quando fomos utilizar o csv como arquivo de base, é recomendavel que seja definido o separador que foi utilizado para gerar o arquivo, no nosso caso foi utilizado o ;, entao passamos o parametro ```sep=';'``` junto com o caminho do arquivo
- Outro ponto que é relevante mencionar no csv, é o header, que por padrão o spark não considera a primeira linha como header, caso seja esse o caso do arquivo que estamos lendo, podemos utilizar o parametro ```header``` definindo ele como true

In [12]:
# Lemos o csv sem utilizar o separador e nem o header
csv_errado = sessao.read.csv('dados/MICRODADOS_ENEM_2019.csv')

In [17]:
csv_errado.printSchema()

root
 |-- _c0: string (nullable = true)



In [4]:
# Lemos o mesmo arquivo, agora utilizando o separador e definindo a primeira linha como header
csv = sessao.read.csv('dados/MICRODADOS_ENEM_2019.csv', sep=';',header=True)

In [16]:
csv.printSchema()

root
 |-- NU_INSCRICAO: string (nullable = true)
 |-- NU_ANO: string (nullable = true)
 |-- CO_MUNICIPIO_RESIDENCIA: string (nullable = true)
 |-- NO_MUNICIPIO_RESIDENCIA: string (nullable = true)
 |-- CO_UF_RESIDENCIA: string (nullable = true)
 |-- SG_UF_RESIDENCIA: string (nullable = true)
 |-- NU_IDADE: string (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: string (nullable = true)
 |-- TP_COR_RACA: string (nullable = true)
 |-- TP_NACIONALIDADE: string (nullable = true)
 |-- CO_MUNICIPIO_NASCIMENTO: string (nullable = true)
 |-- NO_MUNICIPIO_NASCIMENTO: string (nullable = true)
 |-- CO_UF_NASCIMENTO: string (nullable = true)
 |-- SG_UF_NASCIMENTO: string (nullable = true)
 |-- TP_ST_CONCLUSAO: string (nullable = true)
 |-- TP_ANO_CONCLUIU: string (nullable = true)
 |-- TP_ESCOLA: string (nullable = true)
 |-- TP_ENSINO: string (nullable = true)
 |-- IN_TREINEIRO: string (nullable = true)
 |-- CO_ESCOLA: string (nullable = true)
 |-- CO_MUNICIPIO_ESC: 

### Conferindo os dados que foram importados
- Quando importamos os dados é importante termos certeza do tipo de dado que consta no schema que foi importado
- O schema é a estrutura do dataframe, ou seja, o tipo de dado que a coluna tem, podendo ser definido na hora da importação, utilizando o metodo [StructType](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.types.StructType.html) que recebe uma lista de [StructField](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.types.StructField.html). Dessa forma conseguimos ter certeza do tipo de dado que estamos lidando após a importanção. Isso é importante uma vez que queremos jogar o dado para um banco relacional ou para um DW já estruturado, então temos que ter um controle do dado que estamos lidando. Ou um campo de timestamp que queremos lidar apenas com a data e não com todo o conteudo.
- Com isso, após a importação, se quisermeos verificar o que foi importado podemos usar o metodo ```printSchema()``` que retorna a estrutura do dataframe importado.
- Vale notar no exemplo da importação do csv, sem o separador e o header a diferença no schema que é retornado.

### Tipos de APIS

- Dentro do Spark temos atualmente 3 formas de dispor os dados
    - RDD - Resilient Distributed Datasets
         - Dataset distritubuido resiliente
           - Foi o primeiro modelo a ser utilizado no spark, sendo o mais antigo e a estrutura fundamental no funcionamento do spark.
           - A Criação dos RDDs pode ser feita através de paralelização de uma lista por exemplo, conseguimos fazer a transformação de um RDD em um dataframe e da mesma forma de um dataframe em um RDD.
           - O uso de RDD é recomendado quando queremos fazer transformação em um nivel mais baixo no conjunto de dados que temos.
           - O RDD é muito utilizado quando operamos com chave valor, para um conjunto de dados com duas colunas por exemplo.
           - Talvez o principalmente entrave do RDD é a necessidade de ser definido o tipo de dado que estamos lidando
                - Caso o RDD seja formado de um arquivo importado e não de um dataframe, o spark precisa ser informado exatamente cada formato de dado que esta sendo passado para poder operar de acordo.
           - Com um RDD podemos fazer as transformações em conjunto com outros RDDS, ou seja, operações de joins e union
           - Podemos operar com o RDD sozinho, fazendo operações internas nele, como operações matematicas ou de filtros por exemplos
           - O resto das operações possiveis podemos ver na [documentação](https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations)
 - Dataframe
      - O Dataframe resolve alguns dos entraves que o RDD cria no processo de engenharia
      - Podemos acompanhar o processo de tranformação dos dados na medida que ele vai ocorrendo.
      - Conseguimos importar com facilidade arquivos CSV, Parquet, AVRO, HDFS ou HIVE, além de JDBC
      - Utilizando o sparkSQL conseguimos executar operações SQL em cima do dataframe
 - Datasets 
      - Basicamente junta os dois elementos